In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')




pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

postal_code = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
postal_code.columns = ['PostalCode', 'Borough', 'Neighborhood']

postal_code.head(10)

postal_code = postal_code[postal_code.Borough != 'Not assigned']

postal_code.reset_index(drop = True, inplace = True)

for i in range(len(postal_code)):
    
    if postal_code.loc[i, 'Neighborhood'] == 'Not assigned':
    
        postal_code.loc[i, 'Neighborhood'] = postal_code.loc[i, 'Borough']

postal_code.head(10)

output = postal_code.drop_duplicates(['PostalCode', 'Borough']).copy()

output.reset_index(drop = True, inplace = True)

for i in range(len(output)):
    
    nh_List = []
    
    for j in range(len(postal_code)):
    
        if postal_code.loc[j, 'PostalCode'] == output.loc[i, 'PostalCode']:
        
            if postal_code.loc[j, 'Borough'] == output.loc[i, 'Borough']:
            
                nh_List.append(postal_code.loc[j, 'Neighborhood'])
    
    nh_string = nh_List[0]
    
    for k in range(1, len(nh_List)):
    
        nh_string = nh_string + ', ' + nh_List[k]
    
    output.loc[i, 'Neighborhood'] = nh_string

    
output.head(10)

print("Rows: ".format(output.shape[0]))

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-